In [16]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



In [1]:
import ollama

In [ ]:
# Define messages for SmolLM2
messages = [
    {"role": "user", "content": "Hello, how are you?"},
    {
        "role": "assistant",
        "content": "I'm doing well, thank you! How can I assist you today?",
    },
]

In [2]:
import ollama
response = ollama.chat(
    'gemma:2b',
    messages=[{'role' : 'user', 'content': "hey, hello"}],
    )

print(dict(response.message))

{'role': 'assistant', 'content': "Hello! 👋 It's great to hear from you. How can I help you today? 😊", 'images': None, 'tool_calls': None}


The main purpose of apply_chat_template is to format conversations in a way that the model expects, with clear markers for different speakers and message boundaries

In [6]:
ds

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

In [ ]:
def process_dataset(sample):
    # TODO: 🐢 Convert the sample into a chat format
    # use the tokenizer's method to apply the chat template
    return sample


ds = ds.map(process_dataset)

In [8]:
prompt = "Write a poem about programming "

response = ollama.chat(
    'gemma:2b',
    messages=[{'role' : 'user', 'content': prompt}],
    options= {'num_predict':512, 'temperature':0.1}
    )

print(dict(response.message))

{'role': 'assistant', 'content': "The keyboard clicks, the screen blinks bright,\nA symphony of code, a digital sight.\nLines of logic, a puzzle to solve,\nEach character a puzzle piece, to be told.\n\nThe compiler hums, the program takes flight,\nA dance of bits, a digital delight.\nFrom simple scripts, to complex designs,\nThe programmer's spirit, forever shines.\n\nThe debugger's gaze, a mirror's gleam,\nReveals the flaws, the errors that appear.\nWith every fix, a lesson is learned,\nA master of logic, a coder born.\n\nThe programmer's heart, a fire so bright,\nA passion for code, a burning light.\nIn every line they write, a spark is born,\nIgniting the world, a digital morn.\n\nSo let the keys dance, let the code take flight,\nThe programmer's spirit, shines so bright.\nCoding, a journey, a path to explore,\nWhere imagination and code, forever soar.", 'images': None, 'tool_calls': None}


In [9]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [13]:
ds['train']

Dataset({
    features: ['full_topic', 'messages'],
    num_rows: 2260
})

In [30]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "HuggingFaceTB/SmolLM2-135M"

tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)

model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)


In [31]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


In [32]:
import torch
from trl import SFTConfig, SFTTrainer, setup_chat_format
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
      # Set a unique name for your model
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)


Map: 100%|██████████| 119/119 [00:00<00:00, 1367.15 examples/s]
max_steps is given, it will override any value given in num_train_epochs


In [33]:
trainer.train()

  0%|          | 2/1000 [00:25<3:39:22, 13.19s/it]

KeyboardInterrupt: 